# 0. Estructuras utilizadas

In [1]:
from instance import WP
from instance import Visita
import random

La clase Ruta que utilizaremos como representación de una solución factible

In [2]:
class Ruta :
    
    def __init__(self):
        self.vertices = []
        self.tiempo = []  
        self.dist = []  # dist[i] := dist(vertices[i] , vertices[i+1])
        # se lleva la distancia, aun siendo información redundante en la ruta, por simplificar el código posterior
        
    def __str__(self):
        return 'Ruta: ' + str(self.vertices) + '\nTiempos: ' + str(self.tiempo) 

## 0.1 Función fitness


A continuacion definimos la función fitness, que dados una instancia y una ruta, determina el numero de testigos que dicen simultaneamente la verdad. La dividimos en varias funciones:

* visitar : Dada una ruta para cada persona del problema, y el número de vértices de este, genera una lista para cada vértice del grafo, indicando las distintas visitas que ha recibido dicho vértice. La clase Visita incluye:
  - El numero correspondiente al actor que realiza la visita
  - El intervalo [a, b] durante el cual dicha persona está en el vértice



* comprobar_testimonio(visitas, tes) : Esta función devuelve True o False dependiendo de si el testimonio de la entrada es cierto o no, conociendo todas las visitas que se han hecho a cada vértice

* fitness(instancia, rutas) : Devuelve el número de testigos que están diciendo la verdad dada una solución (ruta) del problema.

In [3]:
def fitness(instancia, rutas) :
    # visitar() genera una lista de visitas en cada vértice del grafo, indicando cada una que una persona ha estado en dicho vertice durante un intervalo
    visitas = visitar(instancia.grafo.V, [rutas]) 
    sinceros = 0
    for testigo in instancia.testimonios :
        b = True
        for tes in testigo :
            if not comprobar_testimonio(visitas, tes) : # Si algun testimonio del testigo falla
                b = False
                break # entonces el testigo miente
        sinceros += b ## Si b es True, todos los testimonios del testigo eran ciertos, por lo tanto es sincero
    return sinceros

In [4]:
# Comprueba si 2 intervalos [a1, b1] [a2, b2] se cortan
def intervalos_disjuntos(a1,b1,  a2,b2):
    assert(a2 <= b2)
    return b1 < a2 or b2 < a1


# Comprueba si un testimonio es cierto
def comprobar_testimonio(visitas , tes) :
    for lugar in tes.lugares : # Para cada vertice
        for visita in visitas[lugar] : # Recorremos todas las visitas realizadas a dicho vertice
            if not intervalos_disjuntos(visita.a, visita.b, tes.a, tes.b) : # Si se visito durante el intervalo requerirdo por el testimonio
                for actor in tes.actores : # Comprobamos si la persona que visito el vertice es uno de nuestros actores
                    if actor == visita.persona :
                        return not tes.negado
    return tes.negado

def visitar(V, rutas) :
    visitas = [[] for _ in range(V)]
    
    for persona, ruta in enumerate(rutas) : 
        
        # Marcamos el vertice inicial como visitado en [0, t_0]
        tiempo = ruta.tiempo[0]
        visitas[ruta.vertices[0]].append( Visita(persona, 0 , tiempo))
        
        # Marcamos en cada vertice que ha sido visitado por la persona en el intervalo [first(k) , last(k)]
        for k in range(1, len( ruta.vertices )) :        
            tiempo += ruta.dist[k-1] # tiempo = first(k)
            visitas[ruta.vertices[k]].append( Visita(persona, tiempo , tiempo + ruta.tiempo[k]))
            tiempo += ruta.tiempo[k]  # tiempo = last(k)
            
    
    for vis in visitas :
        vis.sort()

    return visitas

# 1. Algoritmos voraces

A continuación exponemos una serie de algoritmos voraces para obtener soluciones aproximadas del problema con un solo actor, un testimonio por cada testigo y sin testimonios negados. Los algoritmos reciben una lista de testimonios y devuelven una ruta como solución.


## 1.0 Preámbulos

Comenzamos implementando una función que genera la lista de testimonios de una instancia, que tomaremos como input en los algoritmos voraces.

In [5]:
def lista_testimonios(instancia) :
    testimonios = []
    for testigo in instancia.testimonios :
        for T in testigo :
            testimonios.append(T)   
    return testimonios

A continuación implementamos los algoritmos persistentes, que toman un algoritmo voraz y lo ejecutan comenzando desde cada vértice del grafo como vértice inicial. La solución obtenida por estos es necesariamente mayor o igual que la del correspondiente algoritmo utilizado.

In [6]:
def persistente(instancia, testimonios, algoritmo):
    maximo = 0
    ruta = Ruta()
    for v in range(instancia.grafo.V) :
        R = algoritmo(instancia, testimonios, v)
        valor = fitness(instancia, R)
        if valor > maximo :
            ruta = R
            maximo = valor
    return ruta


## 1.1 Voraz por hora de llegada y salida

Los primeros 2 algoritmos que exponemos parten de la misma idea, ordenamos los testimonios bajo un cierto criterio, y el algoritmo da creedibilidad al primer testimonio de la lista descartando los testimonios posteriores que resulten ser incompatibles con los que ya hayamos dado por ciertos.
La primera versión del algoritmo ordenará los testimonios de menor a mayor valor del parámetro a, la hora de llegada al vértice.
La segunda versión los ordena de menor a mayor valor de b, la última hora en la que se puede cumplir el testimonio. Ambas, una vez ordenados llaman a la siguiente función:

In [7]:
# Recibe una lista de testimonios y un vértice inicial, va cogiendo testimonios de la lista mientras pueda hacerlos ciertos

def voraz(grafo, testimonios, src) :
    R = Ruta()
        
    R.vertices.append(src)
    R.tiempo.append(0)
    time = 0
    for T in testimonios :
        distancia = 1000000000
        dst = -1
        for v in T.lugares : # Intentamos hacer cierto el testimonio T con el vertice que nos pille mas cerca
            d = grafo.dist(src, v)
            if time + d <= T.b and d < distancia :
                dst = v
                distancia = d
        if dst != -1 :  # Si dst no es -1 el testimonio se puede hacer cierto
            arrival = max(T.a, time + distancia)

            # La funcion reconstruir utiliza el algoritmo de Floyd para modificar la ruta R de tal forma que 
            # llegue de src a dst en el tiempo arrival - time, esperando si fuera necesario en el ultimo vertice.
            grafo.reconstruir(src, dst, arrival - time , R)
            src = dst
            time = arrival
    return R
            

In [8]:
def vertice_inicial(testimonios):
    for T in testimonios :
        src = random.choice(T.lugares)
        return src

In [9]:
def voraz_hora_de_llegada(instancia, testimonios, src = -1):

    testimonios.sort(key = lambda tes : tes.a) 

    if src == -1 : # Buscamos el testimonio inicial aleatoriamente de tal forma que cumpla el primer testimonio
        src = vertice_inicial(testimonios)
    
    R = voraz(instancia.grafo, testimonios, src)
    
    return R

In [10]:
def voraz_hora_de_salida(instancia, testimonios, src = -1):

    testimonios.sort(key = lambda tes : tes.b) 

    if src == -1 : # Buscamos el testimonio inicial aleatoriamente de tal forma que cumpla el primer testimonio
        src = vertice_inicial(testimonios)
    
    R = voraz(instancia.grafo, testimonios, src)
    
    return R

## 1.2 Voraz por distancia

La siguiente estrategia voraz parte de un vértice arbitrario y va seleccionando en cada paso el testimonio que se puede cumplir en el menor tiempo posible habiendo comenzado en dicho vértice en el tiempo 0.

In [11]:
def voraz_distancia(instancia, testimonios, src = -1):
    grafo = instancia.grafo

    if src == -1 : # Buscamos el testimonio inicial aleatoriamente de tal forma que cumpla el primer testimonio
        src = vertice_inicial(testimonios)

    descartados = [False for _ in testimonios]
    time = 0
    R = Ruta()
    R.vertices.append(src)
    R.tiempo.append(0)
    cambio = True
    while cambio :
        
        minimo = 1000000000

        cambio = False
        for i,T in enumerate(testimonios) :
            if descartados[i] :
                continue

            # Tomamos el vertice mas cercano del testimonio
            vert = min(T.lugares, key = lambda v : grafo.dist(src,v))
            dist = grafo.dist(src,vert)

            if time + dist > T.b : #Si este testimonio es inalcanzable lo descartamos, pues tampoco lo será en el futuro
                descartados[i] = True
            elif max(time + dist, T.a) < minimo: # Si podemos alcanzar este vertice y mejora el actual
                cambio = True
                minimo = max(time + dist, T.a)      # actualizamos minimo y dst
                dst = vert
                test_index = i

        # Una vez encontrado el testimonio que minimiza la distancia reconstruimos la solucion
        if cambio :
            descartados[test_index] = True
            grafo.reconstruir(src,dst, minimo - time, R)
            src = dst
            time = minimo

    return R

## 1.3 Voraz por número de testimonios

Finalmente la última estrategia voraz que implementaremos es la más costosa computacionalmente, encontrándose en $O(N^3)$, y consiste en que a cada paso del algoritmo seleccionamos viajar a un vértice que verifica algún testimonio y que nos permite verificar el mayor número de testimonios a partir de él. La estructura del código es idéntica al anterior, modificando solamente la selección del vértice.

In [12]:
def voraz_testimonios(instancia, testimonios, src = -1):
    grafo = instancia.grafo

    if src == -1 : # Buscamos el testimonio inicial aleatoriamente de tal forma que cumpla el primer testimonio
        src = vertice_inicial(testimonios)

    descartados = [False for _ in testimonios]
    time = 0
    R = Ruta()
    R.vertices.append(src)
    R.tiempo.append(0)
    cambio = True
    while cambio :
        
        maximo = 0

        cambio = False
        for i,T in enumerate(testimonios) :
            if descartados[i] :
                continue
            testimonio_alcanzable = False
            arrival = 1000000000
            # Tomamos el vertice que maximiza los testimonios alcanzables (vert), y el valor de dicho máximo (maximo)
            for vert in T.lugares :
                if time + grafo.dist(src,vert) <= T.b : # Si este vertice es alcanzable comprobamos si mejora el actual
                    testimonio_alcanzable = True
                    llegada_vert = max(T.a, time + grafo.dist(src,vert))
                    alcanzables = testimonios_alcanzables(grafo, testimonios, descartados, vert, llegada_vert)
                    # esta funcion, implementada a continuación, devuelve la cantidad de testimonios alcanzables a posteriori

                    # Si podemos alcanzar este vertice y mejora el actual, reemplazamos el actual por este. En caso de empate tomamos al que se llegue antes
                    if alcanzables > maximo or (alcanzables == maximo and llegada_vert < arrival ):
                        cambio = True
                        maximo = alcanzables
                        dst = vert
                        arrival = llegada_vert
                        test_index = i

                if not testimonio_alcanzable:
                    descartados[i] = True

        # Una vez encontrado el testimonio que minimiza la distancia reconstruimos la solucion
        if cambio :
            descartados[test_index] = True
            grafo.reconstruir(src,dst, arrival - time, R)
            src = dst
            time = arrival

    return R

In [13]:
#La funcion testimonios_alcanzables indica cuantos testimonios de la lista "testimonios" pueden llegar a ser ciertos partiendo del vertice vert a las t

def testimonios_alcanzables(grafo, testimonios, descartados, vert, t) :
    cuenta = 0
    for i,T in enumerate(testimonios) :
        if not descartados[i]:
            for v in T.lugares :
                if t + grafo.dist(vert,v) <= T.b :
                    cuenta += 1
                    break
    return cuenta 

# 2. Algoritmo de búsqueda


Definimos un algoritmo de búsqueda, de coste exponencial, que nos permita encontrar la solución óptima en instancias pequeñas del problema, probando cada ruta posible en dicha instancia. Su coste en el caso peor es de O(V^M), siendo V el número de vértices del grafo y M el testimonio más tardío. Este caso peor sucede cuando se trata de un grafo completo donde todas las distancias son 1, por lo que para encontrar rutas que solucionen el problema basta con comprobar todas las rutas de tamaño tamaño inferior o igual que M. 

In [14]:
def busqueda_rec(R,time, last_v) :
    if time <= 0 :
        return fitness(instancia, R)
    maximo = fitness(instancia, R)
    for arista in instancia.grafo.adyList[last_v] :
        R.vertices.append( arista.vertice )
        R.tiempo.append( 0 )
        R.dist.append( arista.distancia )
        maximo = max(maximo , busqueda_rec(R, time - arista.distancia, arista.vertice) )
        del R.vertices[-1]
        del R.tiempo[-1]
        del R.dist[-1]
    R.tiempo[-1] += 1
    maximo = max( maximo , busqueda_rec(R, time - 1, last_v) )
    R.tiempo[-1] -= 1
    return maximo
    
def busqueda() :   
    R = Ruta()
    maximo = 0
    for vertice in range(instancia.grafo.V) :
        R.vertices.append( vertice )
        R.tiempo.append( 0 )
        maximo = max(maximo , busqueda_rec(R, instancia.M, vertice) )
        del R.vertices[-1]
        del R.tiempo[-1]
    return maximo

Como comentario, mencionar que se puede hacer otro algoritmo de búsqueda que tolere grafos e intervalos de tiempo más grandes, pero sea de orden al menos $O(N!)$, probando para cada posible reordenación de los testimonios si es posible cumplirlos en dicho orden. Esto sin embargo no nos conviene en nuestro estudio, pues preferimos que el número de testimonios sea grande para que haya más rango de diferencias entre las soluciones obtenidas por un algoritmo y por otro y así facilitar las comparaciones, así que el precio a pagar por ello es que el grafo y el tiempo sean pequeños para que no haya una cantidad exagerada de rutas posibles.

# 3. Comparación algoritmos voraces

En esta sección definimos las funciones utilizadas para evaluar y comparar los distintos algoritmos voraces y el algoritmo de búsqueda

## 3.1 Definiciones


Comenzamos creando una lista con todos los algoritmos voraces a ser comparados.

In [15]:
## Definimos una función lambda para poder probar los algoritmos persistentes de la misma forma que los algoritmos voraces
fun = lambda algoritmo : (lambda inst, test : persistente(inst, test, algoritmo) )

voraces = [voraz_hora_de_llegada,
    voraz_hora_de_salida ,
    voraz_distancia , 
    voraz_testimonios,
    fun(voraz_hora_de_llegada), 
    fun(voraz_hora_de_salida), 
    fun(voraz_distancia), 
    fun(voraz_testimonios)]

nombres_voraces = ['Voraz hora de llegada', 
    'Voraz hora de salida', 
    'Voraz distancia', 
    'Voraz numero de testimonios alcanzables',
    'Voraz hora de llegada persistente', 
    'Voraz hora de salida persistente', 
    'Voraz distancia persistente', 
    'Voraz numero de testimonios alcanzables persistente']

Una función test_voraces() que ejecuta todos los métodos voraces para una instancia dada y devuelve sus resultados:

In [16]:
def ejecutar_voraces(instancia):
    
    testimonios = lista_testimonios(instancia)

    soluciones = []

    for i in range(len(voraces)):
        R = voraces[i](instancia, testimonios)
        soluciones.append(R)

    return soluciones

Finalmente, una función que, dado el nombre de un fichero con instancias del problema, ejecuta todos los algoritmos voraces en todas ellas y devuelve los resultados obtenidos por cada algoritmo en una matriz. Creamos asimismo la función análoga que obtiene la solución óptima de las instancias.

In [17]:
def test_voraces(archivo):
    instancia = WP()

    sols = []
    
    with open('test_cases/' +archivo) as f :
        casos = int(f.readline())
        for i in range(casos):
            instancia = WP()
            instancia.leer(f)      # ejecutar_voraces() devuelve rutas, pero solo nos interesa guardar su valor fitness
            rutas = ejecutar_voraces(instancia)
            sols.append( [fitness(instancia, x) for x in rutas ] )

    archivo_out = 'voraces_'+archivo
    with open('test_cases_out/' +archivo_out,'w') as out :
        out.write('Soluciones de los algoritmos geneticos de: ' + archivo + '\n')
        for i in range( len(nombres_voraces) ):
            out.write('\n\n' + nombres_voraces[i] + ':\n')
            for j in range( len(sols) ) :
                out.write(str(sols[j][i]) + ', ')

    return sols

In [18]:
def test_optima(archivo):
    global instancia
    instancia = WP()

    optima = []
    
    with open('test_cases/' + archivo) as f :
        casos = int(f.readline())
        for i in range(casos):
            instancia = WP()
            instancia.leer(f)      
            optima.append( busqueda() )

    archivo_out = 'optimas_'+archivo
    with open('test_cases_out/' + archivo_out,'w') as out :
        out.write('Soluciones optimas de: ' + archivo + '\n')
        for elem in optima :
            out.write(str(elem) + ', ')

    return optima

# 4. Algoritmo genético

## 4.1 Implementación del problema genético en general

En esta sección implementamos un modelo de algoritmo genético que permite resolver cualquier problema de optimización para el que se definan posteriormente funciones poblacion_inicial(), muta(), cruza() y fitness().
Comenzamos creando una clase ProblemaGenetico que utilizaremos como interfaz con los métodos necesarios para resolver un problema genético:

* poblacion_inicial : Genera aleatoriamente una población inicial aleatoria del tamaño pedido que representan posibles soluciones del problema, en nuestro caso, son rutas de una instancia del problema.

* muta : Realiza cambios (mutaciones) al azar sobre un individuo de la población dado, con probabilidad prob, con el objetivo de alcanzar zonas del espacio de búsqueda nuevas, que no quedan cubiertas con los individuos actuales

* cruza : Representa la reproducción de 2 individuos de la población para generar 2 desdencientes con características compartidas con ambos padres.

* fitness : Evalúa un individuo de la población para saber como de "bueno" es como solución de la instancia

El objetivo de crear esta clase, será poder comparar las diferencias en el funcionamiento de distintos métodos de mutación y cruce

In [19]:
class ProblemaGenetico(object):
        def __init__(self, fun_muta , fun_cruza, fun_fitness, fun_pob_inicial, 
        num_generaciones = 100, tam_generacion = 100, proporcion_cruces = 0.7, probabilidad_mutar = 0.1, participantes_torneo = 4):
            self.fun_cruza = fun_cruza
            self.fun_muta = fun_muta
            self.fun_fitness = fun_fitness
            self.fun_pob_inicial = fun_pob_inicial
            self.ngen = num_generaciones
            self.size = tam_generacion
            self.prop_cruces = proporcion_cruces
            self.prob_mutar = probabilidad_mutar
            self.k = participantes_torneo
            self.instancia = None
        
        def poblacion_inicial(self) :
            #  Genera la poblacion inicial aleatoriamente
            return self.fun_pob_inicial(self.instancia, self.size)
        
        def muta(self, individuo):
            #  Devuelve el cromosoma mutado
            return self.fun_muta(self.instancia, individuo, self.prob_mutar)
        
        def cruza(self, padre1, padre2):         
            #  Devuelve el cruce de un par de cromosomas
            return self.fun_cruza(self.instancia, padre1, padre2)
        
        def fitness(self, individuo):    
            #  Función de valoración
            return self.fun_fitness(self.instancia, individuo)

        def solve(self, instancia):
            self.instancia = instancia
            return algoritmo_genetico(self)

A continuación tenemos la implementación del algoritmo genético que utilizaremos para resolver un objeto ProblemaGenetico, y posteriormente la implementación de todas las funciones asociadas a él.

In [20]:
def algoritmo_genetico(prob):
    
    poblacion = prob.poblacion_inicial()    # Creamos la poblacion inicial
    n_padres = round(prob.size * prob.prop_cruces)
    n_padres= int (n_padres if n_padres%2==0 else n_padres-1)
    n_directos = prob.size - n_padres
    maximo = 0
    for _ in range(prob.ngen):    # Creamos ngen generaciones, cada una a partir de la anterior
        poblacion, mejor, valor_mejor = nueva_generacion(prob, poblacion, n_padres, n_directos)
        
        # Acumulamos el maximo de esta generacion y las pasadas
        
        if maximo < valor_mejor:
            maximo = valor_mejor
            individuo = mejor

    return maximo, individuo


In [21]:
def nueva_generacion(prob, poblacion, n_padres, n_directos):
    
    valor_fitness = [prob.fitness(x) for x in poblacion]
    posicion_mejor = valor_fitness.index( max(valor_fitness) )
    
    ## Seleccionamos aleatoriamente los individuos que seran padres de la siguiente generacion, y los que pasan directamente
    
    directos = seleccion_por_torneo(poblacion, valor_fitness, n_directos, prob.k) 
    padres = seleccion_por_torneo(poblacion, valor_fitness, n_padres  , prob.k)
    cruces =  cruza_padres(prob,padres)
    generacion = directos + cruces
    
    ## Mutamos aleatoriamente algunos individuos de la poblacion
    
    resultado_mutaciones = muta_individuos(prob, generacion)
         # siguiente poblacion , mejor individuo          , valor del mejor individuo    
    return resultado_mutaciones, poblacion[posicion_mejor], valor_fitness[posicion_mejor]

El proceso de selección de los mejores individuos de una población tendrá cierta aleatoriedad añadida para evitar caer en máximos locales. Para ello, para elegir a n individuos, realizaremos n torneos distintos entre k participantes tomados al azar cada uno, de entre los cuales se elegirá al mejor.

In [22]:
def seleccion_por_torneo(poblacion, valor_fitness, n, k):
    # Selección por torneo de n individuos de una población. Siendo k el nº de participantes
    # valor_fitness es una lista del mismo tamño que poblacion que contiene el resultado de aplicar la funcion fitness
    seleccionados = []
    p = 0.7
    for i in range(n):
        participantes_index = random.sample( range(len(poblacion)) ,k)
        participantes_index.sort( key = lambda i : valor_fitness[i])
        i = 0
        while i < k - 1 :   # Distribución de probabilidad pseudo geométrica (la geométrica es sobre los infinitos numeros naturales, aquí al último número le damos toda la probabilidad restante)
            if random.uniform(0,1) < p:
                break
            i += 1
        seleccionado = poblacion[participantes_index[i]]

        seleccionados.append(seleccionado)
    return seleccionados 

Finalmente las siguientes dos funciones se encargan de cruzar a todos los padres de dos en dos, y mutar con probabilidad p a cada hijo, para dar lugar a la siguiente generación.

In [23]:
def cruza_padres(problema_genetico,padres):
    l = []
    for i in range(len(padres)//2):
        hijo1, hijo2 = problema_genetico.cruza(padres[2*i],padres[2*i+1])
        l.append(hijo1)
        l.append(hijo2)
        
    return l

def muta_individuos(problema_genetico, poblacion):
    mutaciones = []
    for especimen in poblacion:
        mutaciones.append(problema_genetico.muta(especimen))
    return mutaciones

## 4.2 Función población inicial

Como comentamos en la memoria, probaremos dos métodos distintos para generar la población inicial de un algritmo genético. El primero de ellos crea rutas totalmente al azar, el segundo sin embargo toma las rutas calculadas con los métodos voraces, y las rutas restantes las genera al azar también.
Para generar las rutas aleatorias utilizamos una función ruta_random() que será definida posteriormente en la sección de mutaciones.

In [24]:
MAX_TIEMPO_ESPERA = 4

def poblacion_inicial1(instancia, size):
    poblacion = []
    
    for j in range(size) :
        poblacion.append(Ruta())
        
        ruta_random(instancia.grafo, -1, -1, instancia.M, poblacion[j])

    return poblacion

In [25]:
def poblacion_inicial2(instancia, size):
    assert(size >= len(voraces))
    soluciones_voraces = ejecutar_voraces(instancia)

    ## Arreglamos las soluciones voraces para que sean rutas de tamaño M como requiere el algoritmo genetico
    
    for ruta in soluciones_voraces :
        suma = 0
        for i in range(len(ruta.dist)) :
            suma += ruta.dist[i] + ruta.tiempo[i]
        ruta.tiempo[-1] = instancia.M - suma

    return soluciones_voraces + poblacion_inicial1(instancia, size - len(voraces)) 

## 4.3 Funcion cruzar

Para la función de cruce, comenzamos definiendo una clase iterador sobre una ruta, y además vaya copiando dicha ruta al final de otra ruta pasada como parámetro, lo cuál nos será de gran ayuda para la posterior implementación de las funciones de cruce y mutación. 

In [26]:
class Iterador_ruta :
    
    def __init__(self) :
        self.index = 0
        self.vertice = 0
        self.a = 0
        self.b = 0
        
            
    def iniciar(self, ruta, hijo = None) :
        self.index = 0
        self.vertice = ruta.vertices[0]
        self.a = 0
        self.b = ruta.tiempo[0]
        
        if not (hijo == None) :
            hijo.vertices.append(self.vertice)
            hijo.tiempo.append(ruta.tiempo[self.index])
        
    def siguiente(self, ruta, hijo = None) :
        if self.index >= len(ruta.dist) :
            return False
        self.a = self.b + ruta.dist[self.index]
        self.index += 1
        self.b = self.a + ruta.tiempo[self.index]
        self.vertice = ruta.vertices[self.index]
        
        if not (hijo == None) :
            hijo.vertices.append(self.vertice)
            hijo.tiempo.append(ruta.tiempo[self.index])
            hijo.dist.append(ruta.dist[self.index - 1])
        
        return True

A continuación implementamos los dos tipos de cruces comentados en la memoria.

In [27]:
def cruza1(instancia, p1, p2) :          
    return sucesor(instancia, p1, p2) , sucesor(instancia, p2, p1)

# Cruza 2 rutas para un mismo actor
def sucesor(instancia, p1, p2) :
    grafo = instancia.grafo
    hijo = Ruta()
    it = Iterador_ruta()
    it.iniciar(p1)
    b = True
    while(b and grafo.dist(it.vertice , p2.vertices[-1]) <= instancia.M - it.b ): 
        b = it.siguiente(p1)
    if it.index == 0:
        return p1
    i = random.randint(0 , it.index-1 ) # Tomamos un índice aleatorio que permita realizar el cruce

    it = Iterador_ruta()
    it.iniciar(p1,hijo) # Copiamos en el hijo desde el inicio de p1 hasta el indice i
    while( it.index < i):
        it.siguiente(p1,hijo)
    hora_salida = it.b
    src = it.vertice

    it2 = Iterador_ruta()
    it2.iniciar(p2) #Buscamos el vértice destino
    while( hora_salida + grafo.dist(src, it2.vertice) > it2.b ):
        it2.siguiente(p2)

    grafo.reconstruir(src, it2.vertice , it2.b - hora_salida , hijo) # Añadimos al hijo el camino mas corto de src a dst

    while(it2.siguiente(p2,hijo)): #Copiamos en el hijo el resto de p2
        continue

    return hijo

In [28]:
# Cruza 2 rutas para un mismo actor

def cruza2(instancia, p1, p2) :
    h1 = Ruta()
    h2 = Ruta()

    it1 = Iterador_ruta()
    it1.iniciar(p1,h1)
    it2 = Iterador_ruta()
    it2.iniciar(p2,h2)
    b1 = True
    b2 = True

    while(b1 and b2) :

        if it1.vertice == it2.vertice :
            if not intervalos_disjuntos(it1.a, it1.b ,it2.a, it2.b) :
                # Dentro de eate if, ambas rutas padres coinciden en el mismo vértice a la misma hora

                h1.tiempo[len( h1.tiempo ) - 1] = it2.b - it1.a
                h2.tiempo[len( h2.tiempo ) - 1] = it1.b - it2.a
                
                # Cruzamos a los hijos
                h1, h2 = h2, h1

                b1 = it1.siguiente(p1, h1)
                b2 = it2.siguiente(p2, h2)
                break # Al eliminar esta instrucción, el cruce pasa a ser con varios puntos de corte en lugar de tener uno solo


        if it1.b < it2.b :
            b1 = it1.siguiente(p1, h1)
        else :
            b2 = it2.siguiente(p2, h2)
   
   # Copiamos en los hijos las colas de los padres correspondientes.
    while b1 :
        b1 = it1.siguiente(p1, h1)
    
    while b2 :
        b2 = it2.siguiente(p2, h2)

    return h1, h2

## 4.4 Función mutar

In [29]:
MAX_INTERVALO_MUTACION = 10

def muta(instancia, ruta, prob) :
    if random.uniform(0, 1) > prob :
        return ruta
    if len(ruta.vertices) < 2 :
        sol = Ruta()
        ruta_random(instancia.grafo, -1, -1, instancia.M,sol)
        return sol


    i = random.randint(0, len(ruta.vertices) - 1)
    j = (i + random.randint(1, MAX_INTERVALO_MUTACION) ) % len(ruta.vertices)
    while j == i :
        j = (i + random.randint(1, MAX_INTERVALO_MUTACION) ) % len(ruta.vertices)
    if i < j: # Nos aseguramos de que i sea el menor entre ambos
        return mutar_intervalo(instancia, ruta, i, j)
    else :
        n = len(ruta.vertices)
        primera_mutacion = mutar_intervalo(instancia,ruta, -1, j)
        m = len(primera_mutacion.vertices)
        return  mutar_intervalo(instancia, primera_mutacion, m-(n-i), m)

Nos ayudamos de la función auxiliar mutar_intervalo, que genera una mutación aleatoria en la ruta en los vértices $w_iw_{i+1}\dotsi w_j$.

Además, si $i=-1$ elige el vértice $w_0$ al azar y genera un cambio en los vértices $w_0w_1\dotsi w_j$.

Si $j=-1$, elige el último vértice $w_s$ al azar y modifica el segmento $w_iw_{i-1}\dotsi w_s$.

In [30]:
def mutar_intervalo(instancia, ruta, i, j) :
    mutacion = Ruta()
    
    iterador = Iterador_ruta()  

    if i >= 0 :
        iterador.iniciar(ruta, mutacion)
        ## Dejamos los indices [0, 1,..., i] intactos
        while(iterador.index < i) :
            iterador.siguiente(ruta, mutacion)
        v1 = ruta.vertices[i]
        t1 = iterador.b # tiempo de salida del primer vertice
    else :
        iterador.iniciar(ruta, None)
        v1 = -1 ## indicamos que no hay vertice inicial
        t1 = 0
        
    ## Avanzamos hasta j para saber a que tiempo llega, enviando None para que no añada vertices erroneos a la mutacion
    if j < len(ruta.vertices) :
        while(iterador.index < j) :
            iterador.siguiente(ruta, None)
        v2 = ruta.vertices[j]
        t2 = iterador.a # tiempo de llegada al segundo vertice
    else :
        v2 = -1 ## indicamos que no hay verticee final
        t2 = instancia.M
    
    if t2 - t1 < 0 :
        fehu= 7
     
    ruta_random(instancia.grafo, v1, v2, t2 - t1, mutacion)
    
    ## Dejamos los indices [j,j+1,..., s] del original intactos
    if j < len(ruta.vertices) :
        mutacion.tiempo[-1] += ruta.tiempo[j]
        while(iterador.siguiente(ruta, mutacion)) :
            continue
    return mutacion

In [31]:
# Añade a mutacion una nueva ruta desde src hasta dest

def ruta_random(grafo, src, dest, tiempo, mutacion) :
    
    if src == -1 : # Si el vertice inicial no viene dado, elegimos uno desde que se pueda llegar al destino
        if dest != -1: 
            iniciales = [v for v in range(grafo.V) if grafo.dist(v, dest) <= tiempo]
        else : # Si el destino tampoco se conoce, elegimos un vertice inicial al azar
            iniciales = range(grafo.V)               
        src = random.choice(iniciales)
        mutacion.vertices.append(src)
        mutacion.tiempo.append(0)
        
    if dest == -1 : # Ya teniendo origen, si el destino no se conoce elegimos uno valido
        finales = [v for v in range(grafo.V) if grafo.dist(src, v) <= tiempo]
        if len(finales) == 0:
            mutacion.tiempo[-1] = tiempo
            return
        dest = random.choice(finales)
    
    # Ya teniendo origen y destino, llamamos a la funcion ruta_random antes definida
    
    ruta_random_rec(grafo, src, dest, tiempo, mutacion)

In [32]:
# Añade a mutacion una nueva ruta desde src hasta dest
# No espera tiempo en ningun vertice, salvo en el ultimo

def ruta_random_rec(grafo, src, dest, tiempo, mutacion) :    

    if src == dest :
        mutacion.tiempo[-1] += tiempo
        return mutacion
    # Tomamos por posibles adyacentes los vertices que nos permiten llegar al destino a tiempo
    posibles_adyacentes = [arista for arista in grafo.adyList[src] if arista.distancia + grafo.dist(arista.vertice, dest) <= tiempo]

    arista_elegida = random.choice(posibles_adyacentes)
    
    mutacion.vertices.append( arista_elegida.vertice )
    mutacion.dist.append( arista_elegida.distancia )
    
    # La recursion finaliza cuando llegamos al destino
    if arista_elegida.vertice == dest :
        mutacion.tiempo.append( tiempo - arista_elegida.distancia )
        return 
    else :
        tiempo_restante = tiempo - arista_elegida.distancia - grafo.dist(arista_elegida.vertice , dest)
        espera = espera_tiempo(tiempo_restante)
        mutacion.tiempo.append(espera)
        ruta_random_rec(grafo, arista_elegida.vertice, dest, tiempo - arista_elegida.distancia - espera, mutacion)
    

In [33]:
import numpy

def espera_tiempo(tiempo) :

    # Version 1: distribucion geometrica, devuelve valores z = 1,2,3,... con probabilidad P(z == k) = p*(1-p)^(k-1)
    z = numpy.random.geometric(p=0.35) - 1
    while z > tiempo :
        z = numpy.random.geometric(p=0.35) - 1
    return z


# 5. Comparación algoritmos genéticos

En esta sección definimos las funciones utilizadas para evaluar y comparar varias configuraciones del algoritmo genético

In [34]:
# Configuraciones probadas en los apartados 6.3.1 y 6.3.2 de la memoria

geneticos = [
    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250) ,
    ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250) ,
    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial2, num_generaciones = 1000, tam_generacion = 250) ,
    ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial2, num_generaciones = 1000, tam_generacion = 250) 
]

nombres_geneticos = [
    'ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250)' ,
    'ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250)' ,
    'ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial2, num_generaciones = 1000, tam_generacion = 250)' ,
    'ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial2, num_generaciones = 1000, tam_generacion = 250) '
]

# Configuraciones probadas en el apartado 6.3.3 de la memoria

''' geneticos = [

    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250) ,
    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 500 , tam_generacion = 500) ,
    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 250 , tam_generacion = 1000) ,
    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250, probabilidad_mutar = 0.2) ,
    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 500 , tam_generacion = 500, probabilidad_mutar = 0.2) ,
    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 250 , tam_generacion = 1000, probabilidad_mutar = 0.2) ,

    ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250) ,
    ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 500 , tam_generacion = 500) ,
    ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 250 , tam_generacion = 1000) ,
    ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250, probabilidad_mutar = 0.2) ,
    ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 500 , tam_generacion = 500, probabilidad_mutar = 0.2) ,
    ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 250 , tam_generacion = 1000, probabilidad_mutar = 0.2) ,
]

nombres_geneticos = [

    'ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250)' ,
    'ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 500 , tam_generacion = 500)' ,
    'ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 250 , tam_generacion = 1000)' ,
    'ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250, probabilidad_mutar = 0.2)' ,
    'ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 500 , tam_generacion = 500, probabilidad_mutar = 0.2)' ,
    'ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 250 , tam_generacion = 1000, probabilidad_mutar = 0.2)' ,

    'ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250)' ,
    'ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 500 , tam_generacion = 500)' ,
    'ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 250 , tam_generacion = 1000)' ,
    'ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250, probabilidad_mutar = 0.2)' ,
    'ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 500 , tam_generacion = 500, probabilidad_mutar = 0.2)' ,
    'ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 250 , tam_generacion = 1000, probabilidad_mutar = 0.2)'
] '''

" geneticos = [\n\n    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250) ,\n    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 500 , tam_generacion = 500) ,\n    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 250 , tam_generacion = 1000) ,\n    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250, probabilidad_mutar = 0.2) ,\n    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 500 , tam_generacion = 500, probabilidad_mutar = 0.2) ,\n    ProblemaGenetico(muta, cruza1, fitness, poblacion_inicial1, num_generaciones = 250 , tam_generacion = 1000, probabilidad_mutar = 0.2) ,\n\n    ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 1000, tam_generacion = 250) ,\n    ProblemaGenetico(muta, cruza2, fitness, poblacion_inicial1, num_generaciones = 500 , tam_generaci

In [35]:
def ejecutar_geneticos(instancia):

    soluciones = []

    for i in range(len(geneticos)):
        valor,_ = geneticos[i].solve(instancia)
        soluciones.append(valor)
    return soluciones

def test_geneticos(archivo):
    instancia = WP()

    sols = []
    
    with open('test_cases/' +archivo) as f :
        casos = int(f.readline())
        for i in range(casos):
            instancia = WP()
            instancia.leer(f)   
            sols.append( ejecutar_geneticos(instancia) )

    archivo_out = 'geneticos_'+archivo
    with open('test_cases_out/' +archivo_out,'w') as out :
        out.write('Soluciones de los metodos voraces de: ' + archivo + '\n')
        for i in range( len(nombres_geneticos) ):
            out.write('\n\n' + nombres_geneticos[i] + ':\n')
            for j in range( len(sols) ) :
                out.write(str(sols[j][i]) + ', ')

    return sols

In [36]:
def ejecutar_geneticos_repeticiones(instancia, repeticiones):
    
    soluciones = []

    for i in range(len(geneticos)):
        valores = [ geneticos[i].solve(instancia)[0] for _ in range(repeticiones) ]
        soluciones.append(valores)
    return soluciones

def test_geneticos_repeticiones(archivo, repeticiones):
    instancia = WP()

    sols = []
    
    with open('test_cases/' +archivo) as f :
        casos = int(f.readline())
        for i in range(casos):
            instancia = WP()
            instancia.leer(f)      # ejecutar_voraces() devuelve rutas, pero solo nos interesa guardar su valor fitness
            sols.append( ejecutar_geneticos_repeticiones(instancia , repeticiones) )


    archivo_out = 'geneticos_repeticiones_'+archivo
    with open('test_cases_out/' +archivo_out,'w') as out :
        out.write('Soluciones de los metodos voraces de: ' + archivo + '\n')
        for i in range( len(nombres_geneticos) ):
            out.write('\n\n' + nombres_geneticos[i] + ':\n')
            for j in range( len(sols) ) :
                out.write('\nInstancia ' + str(j) + ':')
                valores = sols[j][i]
                media = sum(valores)/len(valores)
                maximo = max(valores)
                minimo = min(valores)
                varianza =  sum([ (x-media)**2 for x in valores]) / len(valores) 
                out.write(str(sols[j][i]) + '\tmedia: ' + str(media) + '\tmaximo: ' + str(maximo) + '\tminimo: ' + str(minimo) + '\tvarianza: ' + str(varianza))

    return sols

# 6. Pruebas realizadas

Generamos cuatros instancias aleatorias como están explicadas en la memoria, 
* I1 para el apartado 6.3.1.
* I2 e I4 para el apartado 6.3.2
* I3 para el apartado 6.3.3
Además tenemos en la carpeta correspondiente el archivo 'contraejemplo.txt'.

Para poder experimentar en tiempo más breve creamos un archivo con 3 casos de prueba pequeños


In [37]:
from instance import generar_archivo

generar_archivo('prueba.txt' , casos = 3 , vertices = 8, distancia = 8, testigos = 100, testimonios_por_testigo = 1, M = 10, intervalo_testimonio = 4 )

# generar_archivo('I1.txt' , casos = 100 , vertices = 8, distancia = 8, testigos = 100, testimonios_por_testigo = 1, M = 10, intervalo_testimonio = 4 )

# generar_archivo('I2.txt' , casos = 100 , vertices = 20, distancia = 10, testigos = 250, testimonios_por_testigo = 1, M = 100, intervalo_testimonio = 10 )

# generar_archivo('I3.txt' , casos = 50 , vertices = 100, distancia = 10, testigos = 500, testimonios_por_testigo = 1, M = 200, intervalo_testimonio = 10 )

# generar_archivo('I4.txt' , casos = 10 , vertices = 50, distancia = 10, testigos = 200, testimonios_por_testigo = 5, M = 100, intervalo_testimonio = 10,
#                    lugares_por_testimonio = 5, proporcion_testimonios_negados = 0.2)



In [38]:
solucion_voraz = test_voraces('prueba.txt')


In [39]:
solucion_optima = test_optima('prueba.txt')

In [40]:
solucion_geneticos = test_geneticos('prueba.txt')   # 2 minutos

In [41]:
solucion_geneticos_repeticiones = test_geneticos_repeticiones('prueba.txt',10)